In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# from torch.utils.data import Dataset, DataLoader
# from transformers import BertForTokenClassification, BertForSequenceClassification,BertPreTrainedModel, BertModel
# from transformers import AutoTokenizer, BertTokenizer
# import torch.nn as nn
# import torch
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from scipy.optimize import minimize

import pickle

In [ ]:
# class BertForRegression(BertPreTrainedModel):
#     def __init__(self, config,params):
#         super().__init__(config)
#         self.bert = BertModel(config)
#         self.dropout = nn.Dropout(params['dropout'])
#         self.classifier = nn.Linear(config.hidden_size, 1)
#         self.loss_fct = nn.MSELoss()
#         self.init_weights()

#     def forward(
#             self,
#             input_ids=None,
#             attention_mask=None,
#             token_type_ids=None,
#             position_ids=None,
#             head_mask=None,
#             inputs_embeds=None,
#             labels=None,
#     ):
#         outputs = self.bert(
#             input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
#             position_ids=position_ids,
#             head_mask=head_mask,
#             inputs_embeds=inputs_embeds,
#         )
#         pooled_output = outputs[1]

#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)

#         outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

#         if labels is not None:
#             loss = self.loss_fct(logits, labels)
#             outputs = (loss,) + outputs

#         return outputs  # (loss), logits, (hidden_states), (attentions)

    
    



In [ ]:
# class Triage(Dataset):
#     def __init__(self, dataframe, tokenizer, max_len):
#         self.len = len(dataframe)
#         self.data = dataframe
#         self.tokenizer = tokenizer
#         self.max_len = max_len
        
#     def __getitem__(self, index):
#         title = str(self.data.text[index])
#         title = " ".join(title.split())
#         inputs = self.tokenizer.encode_plus(
#             title,
#             None,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             padding='max_length',
#             return_token_type_ids=True,
#             truncation=True
#         )
#         ids = inputs['input_ids'] 
#         mask = inputs['attention_mask']

#         return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'targets': torch.tensor(self.data.labels[index], dtype=torch.float32)
#         } 
    
#     def __len__(self):
#         return self.len

In [ ]:
# def predict(model, dataloader, device):
#     predicted_label = []
#     actual_label = []
#     model.eval()
#     with torch.no_grad():
#         for step,data in tqdm(enumerate(dataloader, 0), total=len(dataloader)):
#             input_ids = data['ids'].to(device, dtype = torch.long)
#             attention_mask = data['mask'].to(device, dtype = torch.long)
#             targets = data['targets'].to(device, dtype = torch.float32)
#             targets = targets.unsqueeze(1)

# #             input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
#             output = model(input_ids, attention_mask)
                        
#             predicted_label += output[0]
#             actual_label += targets
            
#     return predicted_label

In [ ]:
# from torch import cuda
# device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# # Defining some key variables that will be used later on in the training
# MAX_LEN = 256 
# VALID_BATCH_SIZE = 32
# MODEL_PATHS = [
#     '../input/bert-base-uncased-reddit',
# #     '../input/bert-base-uncased-founta',
#     '../input/bert-base-uncased-toxic-comment',
# #     '../input/bert-base-uncased-toxic-unintended'
#     '../input/bert-base-uncased-davidson'
# ]

filename = '../input/tf-idf-ruddit/finalized_model.sav'
tfidf_reddit = pickle.load(open(filename, 'rb'))

filename = '../input/tfidf-balanced/finalized_model.sav'
tfidf = pickle.load(open(filename, 'rb'))

filename = '../input/notebook907c10f9d0/finalized_model.sav'
tfidf_x = pickle.load(open(filename, 'rb'))

In [ ]:
# def Prediction(model_path, test_df):
#     tokenizer = BertTokenizer.from_pretrained(model_path)
#     test = pd.DataFrame()
#     test['text'] = test_df.copy()
#     test['labels'] = 0
    
 
#     testing_set = Triage(test, tokenizer, MAX_LEN)


#     test_params = {'batch_size': 32,
#                     'shuffle': False,
#                     'num_workers': 0
#                     }

#     test_loader = DataLoader(testing_set, **test_params)
    
#     model = BertForRegression.from_pretrained(
#             model_path, # Use the 12-layer BERT model, with an uncased vocab
#             params={'dropout':0.2}).to(device)
    
# #     model.eval()
    
#     output = predict(model, test_loader, device)
    
#     out2 =[]
#     for out in output:
#         out2.append(out.cpu().detach().numpy())

#     out = np.array(out2).reshape(len(out2))
    
#     return out
    

In [ ]:
import re
from collections import defaultdict

special_character_removal=re.compile(r'[^?!.,:a-z\d ]',re.IGNORECASE)
# regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)
word_count_dict = defaultdict(int)
toxic_dict = {}

def clean_text(text, remove_stopwords=False, stem_words=False, count_null_words=True, clean_wiki_tokens=True):
    # Clean the text, with the option to remove stopwords and to stem words.
    # dirty words
    #text = text.lower()
    text = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", "", text)
    text = re.sub(r"(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}", "", text)
    
    if clean_wiki_tokens:
        # Drop the image
        text = re.sub(r"image:[a-zA-Z0-9]*\.jpg", " ", text)
        text = re.sub(r"image:[a-zA-Z0-9]*\.png", " ", text)
        text = re.sub(r"image:[a-zA-Z0-9]*\.gif", " ", text)
        text = re.sub(r"image:[a-zA-Z0-9]*\.bmp", " ", text)

        # Drop css
        text = re.sub(r"#([A-Fa-f0-9]{6}|[A-Fa-f0-9]{3})", " ",text)
        text = re.sub(r"\{\|[^\}]*\|\}", " ", text)
        
        # Clean templates
        text = re.sub(r"\[?\[user:.*\]", " ", text)
        text = re.sub(r"\[?\[user:.*\|", " ", text)        
        text = re.sub(r"\[?\[wikipedia:.*\]", " ", text)
        text = re.sub(r"\[?\[wikipedia:.*\|", " ", text)
        text = re.sub(r"\[?\[special:.*\]", " ", text)
        text = re.sub(r"\[?\[special:.*\|", " ", text)
        text = re.sub(r"\[?\[category:.*\]", " ", text)
        text = re.sub(r"\[?\[category:.*\|", " ", text)
    
   
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"\!", " ! ", text)
    text = re.sub(r"\"", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = replace_numbers.sub(' ', text)
    #text = special_character_removal.sub('',text)

    if count_null_words:
        text = text.split()
        for t in text:
            word_count_dict[t] += 1
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)

    return (text)



In [ ]:
# # tokenizer = AutoTokenizer.from_pretrained(MODEL_PATHS)
val_df = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
val_df.drop_duplicates(subset='less_toxic', keep='first', inplace=True)
val_df.reset_index(inplace=True)
val_df.head()
############################
# val_df = val_df.head(1000) #

In [ ]:
# val_df.head()

In [ ]:
model_predictions_less=[]
model_predictions_more=[]


# for model_path in MODEL_PATHS:
#     print(model_path)
#     mod1_less_tox = Prediction(model_path, val_df['less_toxic'])
#     mod1_more_tox = Prediction(model_path, val_df['more_toxic'])
#     model_predictions_less.append(mod1_less_tox)
#     model_predictions_more.append(mod1_more_tox)

In [ ]:
model_predictions_less.append(tfidf_reddit.predict(val_df['less_toxic']))
model_predictions_more.append(tfidf_reddit.predict(val_df['more_toxic']))

val_df['less_toxic'] = [clean_text(text) for text in val_df['less_toxic']]
val_df['more_toxic'] = [clean_text(text) for text in val_df['more_toxic']]

model_predictions_less.append(tfidf.predict(val_df['less_toxic']))
model_predictions_less.append(tfidf_x.predict(val_df['less_toxic']))

model_predictions_more.append(tfidf.predict(val_df['more_toxic']))
model_predictions_more.append(tfidf_x.predict(val_df['more_toxic']))

In [ ]:
val_pred_less_tox = np.concatenate([mod1_less_tox[:, None] for mod1_less_tox in model_predictions_less], axis=1)
val_pred_more_tox = np.concatenate([mod1_more_tox[:, None] for mod1_more_tox in model_predictions_more], axis=1)

In [ ]:
for i in range(len(model_predictions_less)):
    print(np.round((model_predictions_less[i] < model_predictions_more[i]).mean(),2))



In [ ]:
def objective(weights):
    """ Calculate the score of a weighted average of predictions
    
    Parameters
    ----------
    weights: array
        the weights applied to the average of the base predictions
        
    Returns
    -------
    float
        The mean_squared_error score of the ensemble
    """
    
    y_ens_less_tox = np.average(val_pred_less_tox, axis=1, weights=weights)
    y_ens_more_tox = np.average(val_pred_more_tox, axis=1, weights=weights)
    return np.round((y_ens_less_tox < y_ens_more_tox).mean(),2)



In [ ]:
print('\n Finding Blending Weights ...')
res_list = []
weights_list = []

for k in range(100):
    starting_values = np.random.uniform(size=val_pred_less_tox.shape[1])
    bounds = [(0, 1)] * val_pred_less_tox.shape[1]
    
    res = minimize(objective,
                   starting_values,
                   method='L-BFGS-B',
                   bounds=bounds,
                   options={'disp': False,
                            'maxiter': 100000}) 
    
    res_list.append(res['fun'])
    weights_list.append(res['x'])
    
    print('{iter}\tScore: {score}\tWeights: {weights}'.format(
        iter=(k + 1),
        score=res['fun'],
        weights='\t'.join([str(item) for item in res['x']])))

    
bestSC   = np.max(res_list)
bestWght = weights_list[np.argmax(res_list)]
weights  = bestWght
blend_score = round(bestSC, 6)

In [ ]:
print('\n Ensemble Score: {best_score}'.format(best_score=bestSC))
print('\n Best Weights: {weights}'.format(weights=bestWght))

In [ ]:
test_df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
# test_df = test_df.head(500)

In [ ]:
model_predictions=[]

# for model_path in MODEL_PATHS:
#     mod1_tox = Prediction(model_path, test_df['text'])
#     model_predictions.append(mod1_tox)

In [ ]:

model_predictions.append(tfidf_reddit.predict(test_df['text']))

test_df['text'] = [clean_text(text) for text in test_df['text']]

model_predictions.append(tfidf.predict(test_df['text']))
model_predictions.append(tfidf_x.predict(test_df['text']))

In [ ]:
test_pred_tox = np.concatenate([model_pred[:, None] for model_pred in model_predictions], axis=1)


In [ ]:
# best_weights = [0.76944408, 0.39145082, 0.28043746, 0.90303933]

In [ ]:
def final_predictions(test_preds, best_weights):
    """ Calculate the score of a weighted average of predictions
    
    Parameters
    ----------
    weights: array
        the weights applied to the average of the base predictions
        
    Returns
    -------
    float
        The mean_squared_error score of the ensemble
    """
    
    output_y = np.average(test_preds, axis=1, weights=best_weights)
    return output_y
    

In [ ]:
output=final_predictions(test_pred_tox, bestWght)



In [ ]:
from scipy.stats import rankdata

out2 =[]
for out in output:
    out2.append(out)

out = np.array(out2).reshape(len(out2))

submission = pd.DataFrame({'comment_id': test_df['comment_id'], 'score':out})
submission['score'] = rankdata(submission['score'], method='ordinal')
submission.to_csv('submission.csv', index=False)

In [ ]:
submission